In [64]:
import pandas as pd
import numpy as np
import json
import pymysql
from sqlalchemy import create_engine
from datetime import datetime

In [65]:
#Product_summary = pd.read_csv('Product_summary.csv')

In [66]:
Sales_summary = pd.read_csv('Sales_summary_final.csv',low_memory=False)

In [67]:
Sales_summary.head(5)

,title,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country,url,rating,review_count,price,product_type,total_sale,ASIN
0,Fujifilm Instax Mini 8 Instant Film Camera (Ra...,78581587,22726,4.0,09-12-2019 12:50,12680.0,France,https://www.amazon.com/Fujifilm-Instax-Mini-Ra...,4.6,15513.0,52.95,dslr camera,211.80,B00U2UZ40Y
1,Lowepro Photo Hatchback 16L Camera Backpack - ...,78581587,22899,6.0,09-12-2019 12:50,12680.0,France,https://www.amazon.com/Lowepro-Photo-Hatchback...,4.5,1473.0,87.49,dslr camera,524.94,B008O8EGOQ
2,Neewer NW320 Mini TTL Speedlite Flash Automati...,78581586,22061,8.0,09-12-2019 12:49,13113.0,DE,https://www.amazon.com/Neewer-Display-Speedlit...,4.2,611.0,69.99,dslr camera,559.92,B00VE5HQ4I
3,Lowepro Fastpack 350 DSLR Camera Backpack,78581586,23275,24.0,09-12-2019 12:49,13113.0,DE,https://www.amazon.com/Lowepro-Fastpack-DSLR-C...,4.4,1608.0,469.99,dslr camera,11279.76,B000YIWR8G
4,Fujifilm Instax Mini 8 Instant Film Camera (Ra...,78581585,22726,8.0,09-12-2019 12:31,15804.0,DE,https://www.amazon.com/Fujifilm-Instax-Mini-Ra...,4.6,15513.0,52.95,dslr camera,423.60,B00U2UZ40Y


In [73]:
Sales_summary.dropna(inplace = True)

In [69]:
Sales_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554431 entries, 0 to 554430
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   title         554430 non-null  object 
 1   InvoiceNo     554430 non-null  object 
 2   StockCode     554430 non-null  object 
 3   Quantity      554430 non-null  float64
 4   InvoiceDate   554430 non-null  object 
 5   CustomerID    415115 non-null  float64
 6   Country       554430 non-null  object 
 7   url           554417 non-null  object 
 8   rating        554417 non-null  float64
 9   review_count  554417 non-null  float64
 10  price         554417 non-null  float64
 11  product_type  554417 non-null  object 
 12  total_sale    554417 non-null  float64
 13  ASIN          546019 non-null  object 
dtypes: float64(6), object(8)
memory usage: 59.2+ MB


In [70]:
Sales_summary.dtypes

title            object
InvoiceNo        object
StockCode        object
Quantity        float64
InvoiceDate      object
CustomerID      float64
Country          object
url              object
rating          float64
review_count    float64
price           float64
product_type     object
total_sale      float64
ASIN             object
dtype: object

In [71]:
Sales_summary['InvoiceDate'] = Sales_summary['InvoiceDate'].astype('datetime64[ns]')

In [74]:
#import timestamp
#Sales_summary['invoice_date'] = Sales_summary['InvoiceDate'].apply(lambda x: fromtimestamp(x['date']))
Sales_summary['invoice_date'] = [d.date() for d in Sales_summary['InvoiceDate']]
Sales_summary["invoice_time"] = [d.time() for d in Sales_summary["InvoiceDate"]]

In [81]:
Sales_summary['invoice_date'] = Sales_summary['invoice_date'].astype('datetime64[ns]')

In [82]:
Sales_cancelled = pd.read_csv('cancelled.csv',low_memory=False)

In [83]:
Sales_cancelled

,title,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country,url,rating,review_count,price,product_type,total_sale
0,"Magnetic Phone Car Mount, Penom Cell Phone Hol...",C78581569,84978,-1,09-12-2019 11:58,17315.0,DE,https://www.amazon.com/magn%C3%A9tico-universa...,4.7,6561,13.99,smartphone,-13.99
1,ASUS VivoBook S17 S712 Thin and Light 17.3â€ ...,C78581569,20979,-5,09-12-2019 11:58,17315.0,DE,https://www.amazon.com/ASUS-VivoBook-i7-10510U...,3.6,1614,799.99,processor,-3999.95
2,"iHome Bluetooth Mac Mouse with Scroll Wheel, 3...",C78581568,21258,-5,09-12-2019 11:57,15311.0,DE,https://www.amazon.com/iHome-Bluetooth-Mac-Mou...,4.2,714,29.99,keyboard,-149.95
3,"iHome Bluetooth Mac Mouse with Scroll Wheel, 3...",C78581568,21258,-5,09-12-2019 11:57,15311.0,DE,https://www.amazon.com/iHome-Bluetooth-Mac-Mou...,4.2,714,29.99,mouse,-149.95
4,Corsair Hydro Series H60 AIO Liquid CPU Cooler...,C78581499,M,-1,09-12-2019 10:28,15498.0,DE,https://www.amazon.com/CORSAIR-HYDRO-Refrigera...,4.4,3686,79.99,processor,-79.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9641,Fellowes 9112101 Crystals Gel Mouse Pad and Wr...,C78536391,21484,-12,01-12-2018 10:24,17548.0,DE,https://www.amazon.com/Fellowes-9112101-Crysta...,4.1,2310,17.95,mouse,-215.40
9642,NETGEAR 4-Stream Wifi 6 Router (RAX15) - AX180...,C78536391,22557,-12,01-12-2018 10:24,17548.0,DE,https://www.amazon.com/NETGEAR-4-Stream-Router...,4.1,3895,96.26,processor,-1155.12
9643,Google - Pixel 3a with 64GB Memory Cell Phone ...,C78536391,22553,-24,01-12-2018 10:24,17548.0,DE,https://www.amazon.com/Google-Pixel-Memory-Pho...,4.3,3917,349.98,smartphone,-8399.52
9644,Neewer 85mm f/1.8 Manual Focus Aspherical Medi...,C78536383,35004C,-1,01-12-2018 09:49,15311.0,DE,https://www.amazon.com/Neewer-Multi-Coated-Por...,4.2,217,99.99,dslr camera,-99.99


In [91]:
#Sales_summary['invoice_date'] =  pd.to_datetime(Sales_summary['invoice_date'], format='%d/%m/%Y')
Sales_summary['invoice_date'] = pd.to_datetime(Sales_summary['invoice_date'])
Sales_summary['invoice_date'] = Sales_summary['invoice_date'].dt.strftime('%Y-%m-%d')

In [94]:
Sales_summary.head(5)

,title,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country,url,rating,review_count,price,product_type,total_sale,ASIN,invoice_date,invoice_time
0,Fujifilm Instax Mini 8 Instant Film Camera (Ra...,78581587,22726,4.0,2019-09-12 12:50:00,12680.0,France,https://www.amazon.com/Fujifilm-Instax-Mini-Ra...,4.6,15513.0,52.95,dslr camera,211.80,B00U2UZ40Y,2019-12-09,12:50:00
1,Lowepro Photo Hatchback 16L Camera Backpack - ...,78581587,22899,6.0,2019-09-12 12:50:00,12680.0,France,https://www.amazon.com/Lowepro-Photo-Hatchback...,4.5,1473.0,87.49,dslr camera,524.94,B008O8EGOQ,2019-12-09,12:50:00
2,Neewer NW320 Mini TTL Speedlite Flash Automati...,78581586,22061,8.0,2019-09-12 12:49:00,13113.0,DE,https://www.amazon.com/Neewer-Display-Speedlit...,4.2,611.0,69.99,dslr camera,559.92,B00VE5HQ4I,2019-12-09,12:49:00
3,Lowepro Fastpack 350 DSLR Camera Backpack,78581586,23275,24.0,2019-09-12 12:49:00,13113.0,DE,https://www.amazon.com/Lowepro-Fastpack-DSLR-C...,4.4,1608.0,469.99,dslr camera,11279.76,B000YIWR8G,2019-12-09,12:49:00
4,Fujifilm Instax Mini 8 Instant Film Camera (Ra...,78581585,22726,8.0,2019-09-12 12:31:00,15804.0,DE,https://www.amazon.com/Fujifilm-Instax-Mini-Ra...,4.6,15513.0,52.95,dslr camera,423.60,B00U2UZ40Y,2019-12-09,12:31:00


In [95]:
db_data = 'mysql+mysqldb://' + 'root' + ':' + 'arunrock9899100' + '@' + 'localhost' + ':3306/' \
       + 'AMAZON' + '?charset=utf8mb4'

In [96]:
engine = create_engine(db_data)

In [97]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='arunrock9899100',
                         db='AMAZON')
# create cursor
cursor=connection.cursor()
# Execute the to_sql for writting DF into SQL
Sales_summary.to_sql('sales', engine, if_exists='append', index=False) 



In [18]:
Sales_cancelled.to_sql('Sales_cancelled', engine, if_exists='append', index=False) 

C:\Users\ARUNRAJ\anaconda3\lib\site-packages\pandas\io\sql.py:1423: UserWarning: The provided table name 'Sales_cancelled' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [98]:
engine.dispose()
connection.close()